In [ ]:
# install esssential libraries
pip install pandas numpy matplotlib seaborn networkx openpyxl pydeseq2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 6.9 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8023 sha256=945e06f28a44f464e796d601c8e392eaabe06dac611b932f7d3cbcae95e216cf
  Stored in directory: /root/.cache/pip/wheels/4e/56/35/a748fc57279a4b84d0b332879445fed1ad8478e7257986b015
Successfully built session-info


In [ ]:
import pandas as pd

# import DeseqDataSet and DeseqStats from pydeseq2 library to perform differential expression analysis
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

# Class {DESeq2Pipeline}: Automating Differential Expression Analysis
class DESeq2Pipeline:
    def __init__(self, count_file, metadata_file):
        self.count_file = count_file  # 'count_file' is csv file containing gene expression data
        self.metadata_file = metadata_file # The 'metadata_file' is an Excel ('.xlsx') file that contains patient information, patient IDs and their cancer status.
        self.count_matrix = None
        self.metadata = None
        self.results_df = None

    def load_data(self):
        """Loads count matrix and metadata."""
        self.count_matrix = pd.read_csv(self.count_file, index_col=0).astype(int)
        self.metadata = pd.read_excel(self.metadata_file, index_col=0)
        print("Data loaded successfully.")

    def run_deseq2(self):
        """Runs DESeq2 differential expression analysis."""
        dds = DeseqDataSet(
            counts=self.count_matrix.T,
            metadata=self.metadata,
            design_factors="Condition"
        )
        dds.deseq2()
        stat_res = DeseqStats(dds, contrast=['Condition', 'Tumor', 'Normal'])
        stat_res.summary()
        self.results_df = stat_res.results_df.sort_values("padj")
        print("DESeq2 analysis completed.")

    def save_results(self, output_file="deseq2_results.xlsx"):
        """Saves the DESeq2 results to an Excel file."""
        if self.results_df is not None:
            self.results_df.to_excel(output_file)
            print(f"Results saved to {output_file}")
        else:
            print("No results to save!")

In [ ]:
# Initialize the DESeq2Pipeline
pipeline = DESeq2Pipeline("prostate_exp_cancer_preprocess.csv", "prostate_metadata.xlsx")

# Load the expression data and metadata and run the pipeline
pipeline.load_data()
pipeline.run_deseq2()

# save the results to an output file
pipeline.save_results()

Data loaded successfully.


<ipython-input-2-52a4816cd7ce>:21: DeprecationWarning: design_factors is deprecated and will soon be removed.Please consider providing a formulaic formula using the design argumentinstead.
  dds = DeseqDataSet(
Fitting size factors...


Using None as control genes, passed at DeseqDataSet initialization


... done in 0.59 seconds.

Fitting dispersions...
... done in 47.19 seconds.

Fitting dispersion trend curve...
/usr/local/lib/python3.11/dist-packages/pydeseq2/dds.py:804: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
... done in 0.37 seconds.

Fitting MAP dispersions...
... done in 63.53 seconds.

Fitting LFCs...
... done in 12.94 seconds.

Calculating cook's distance...
... done in 0.92 seconds.

Replacing 0 outlier genes.

Running Wald tests...
... done in 4.40 seconds.



Log2 fold change & Wald test p-value: Condition Tumor vs Normal
            baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
ARHGEF10L   8.869308       -0.007911  0.043662 -0.181181  0.856226  0.999988
HIF3A       5.170882        0.013939  0.057358  0.243020  0.807990  0.999988
RNF10      11.736478        0.013785  0.037964  0.363103  0.716528  0.999988
RNF11      10.684086        0.000600  0.039789  0.015073  0.987974  0.999988
RNF13      10.060199        0.003129  0.041000  0.076328  0.939158  0.999988
...              ...             ...       ...       ...       ...       ...
PTRF       11.856192        0.019727  0.037782  0.522131  0.601579  0.999988
BCL6B       6.180026       -0.023485  0.052333 -0.448764  0.653602  0.999988
GSTK1      10.164046        0.003823  0.040793  0.093727  0.925326  0.999988
SELP        7.215098        0.012462  0.048422  0.257359  0.796901  0.999988
SELS       10.053169        0.011117  0.041016  0.271041  0.786359  0.999988

[16278 rows

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

class OmicsVisualizer:
    def __init__(self, deseq2_file, exp_data_file, metadata_file):

        # Load DESeq2 differential expression analysis results
        self.deseq2_results = pd.read_excel(deseq2_file)

        # Load gene expression data from a CSV file
        self.exp_data = pd.read_csv(exp_data_file)

        # Load Meta data excel file
        self.metadata = pd.read_excel(metadata_file)

        # set default log2 fold change value for filtering differentially expressed genes and p value for statistical significance
        self.log2FC_threshold = 0
        self.pval_threshold = 0.05

    def volcano_plot(self):
        """Generates a volcano plot for DESeq2 results."""
        self.deseq2_results["-log10(pvalue)"] = -np.log10(self.deseq2_results["pvalue"])
        self.deseq2_results["Significance"] = "Not Significant"

        # classification of genes to upregulated or down regulated based on threshold cutoffs
        self.deseq2_results.loc[(self.deseq2_results["log2FoldChange"] > self.log2FC_threshold) & (self.deseq2_results["pvalue"] < self.pval_threshold), "Significance"] = "Upregulated"
        self.deseq2_results.loc[(self.deseq2_results["log2FoldChange"] < -self.log2FC_threshold) & (self.deseq2_results["pvalue"] < self.pval_threshold), "Significance"] = "Downregulated"

        plt.figure(figsize=(8, 6))
        sns.scatterplot(data=self.deseq2_results, x="log2FoldChange", y="-log10(pvalue)", hue="Significance",
                        palette={"Upregulated": "red", "Downregulated": "blue", "Not Significant": "gray"}, alpha=0.6)

        # adding vertical and horizontal threshold lines
        plt.axhline(-np.log10(self.pval_threshold), color='black', linestyle='--', linewidth=1)
        plt.axvline(-self.log2FC_threshold, color='black', linestyle='--', linewidth=1)
        plt.axvline(self.log2FC_threshold, color='black', linestyle='--', linewidth=1)
        plt.xlabel("Log2 Fold Change")
        plt.xlim(-1, 1)
        plt.xticks(np.arange(-1, 1.25, 0.25))
        plt.ylabel("-log10 (P-value)")
        plt.title("Volcano Plot of Differential Expression")
        plt.legend(title="Gene Regulation", loc="upper right")
        plt.show()

    def heatmap(self):
        """Generates a heatmap of top 25 differentially expressed genes."""

        # Extract expression data for the top 25 genes
        top_25_genes = self.deseq2_results.nlargest(25, 'log2FoldChange')["Unnamed: 0"]
        top_exp_data = self.exp_data.set_index(self.exp_data.columns[0]).loc[top_25_genes]
        plt.figure(figsize=(12, 8))
        sns.heatmap(top_exp_data, cmap="coolwarm", linewidths=0.5)
        plt.title("Heatmap of Top 25 Differentially Expressed Genes")
        plt.xlabel("Patients")
        plt.ylabel("Genes")
        plt.xticks(rotation=90)
        plt.yticks(rotation=0)
        plt.show()

    def correlation_matrix(self,top_n):
        """Generates a correlation matrix heatmap for top N differentially expressed genes."""

        # Select the top N differentially expressed genes based on the highest log2 fold change
        top_genes = self.deseq2_results.nlargest(top_n, 'log2FoldChange')["Unnamed: 0"]
        filtered_exp_data = self.exp_data.set_index(self.exp_data.columns[0]).loc[top_genes]
        correlation_matrix = filtered_exp_data.T.corr()

        plt.figure(figsize=(12, 10))
        sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".2f")
        plt.title(f'Correlation Matrix of Top {top_n} Differentially Expressed Genes')
        plt.xticks(rotation=45, fontsize=8)
        plt.yticks(fontsize=8)
        plt.show()

    def circular_plot(self, min_cutoff, max_cutoff):
        """Generates a circular plot showing relationships between genes based on a user-defined correlation range."""
        top_25_genes = self.deseq2_results.nlargest(25, 'log2FoldChange')["Unnamed: 0"]
        top_exp_data = self.exp_data.set_index(self.exp_data.columns[0]).loc[top_25_genes]
        top_exp_data_corr = top_exp_data.T.corr()

        gene_graph = nx.Graph()

        # Adding edges only for correlations within the specified range
        for i in range(len(top_exp_data_corr.columns)):
            for j in range(i + 1, len(top_exp_data_corr.columns)):
                gene1 = top_exp_data_corr.columns[i]
                gene2 = top_exp_data_corr.columns[j]
                correlation = top_exp_data_corr.iloc[i, j]
                if min_cutoff < correlation < max_cutoff:
                    gene_graph.add_edge(gene1, gene2, weight=correlation)

        fig, ax = plt.subplots(figsize=(10, 10))
        pos = nx.circular_layout(gene_graph)
        edge_weights = [d['weight'] for u, v, d in gene_graph.edges(data=True)]
        normalized_weights = (np.array(edge_weights) - min(edge_weights)) / (max(edge_weights) - min(edge_weights))
        colors = [plt.cm.coolwarm(w) for w in normalized_weights]
        widths = [abs(d['weight']) * 1.5 for u, v, d in gene_graph.edges(data=True)]

        nx.draw(gene_graph, pos, with_labels=True, node_color="skyblue", node_size=500, font_size=8,
                edge_color=colors, width=widths)

        plt.title(f"Circular Plot of Top 25 DEGs Relationships (Correlation Between {min_cutoff} and {max_cutoff})")
        sm = plt.cm.ScalarMappable(cmap=plt.cm.coolwarm, norm=plt.Normalize(vmin=min(edge_weights), vmax=max(edge_weights)))
        sm.set_array([])
        plt.colorbar(sm, label="Correlation", ax=ax)
        plt.show()


In [ ]:
visualizer = OmicsVisualizer("deseq2_results.xlsx", "prostate_exp_cancer_preprocess.csv", "prostate_metadata.xlsx")
visualizer.volcano_plot()
visualizer.heatmap()
visualizer.correlation_matrix()

In [ ]:
# User input for top N genes in correlation matrix
top_n = int(input("Enter the number of top differentially expressed genes for correlation matrix: "))
visualizer.correlation_matrix(top_n)

# User input for circular plot correlation range
min_cutoff = float(input("Enter the minimum correlation cutoff for circular plot: "))
max_cutoff = float(input("Enter the maximum correlation cutoff for circular plot: "))
visualizer.circular_plot(min_cutoff, max_cutoff)